![](http://observatoriooceanografico.org/wp-content/uploads/elementor/thumbs/logo-o2-2-p31vla9ib17i85olifd4l1xk0e5lktrkal6u0s4c6o.png)<br>
# <center>Mapa do Brasil em relevo</center><br>

Criado por [A.L. Belém](mailto:andrebelem@id.uff.br) @ [Observatório Oceanográfico](http://www.observatoiriooceanografico.uff.br) 2021 (versão 1) para disciplina de **Mudanças Climáticas e seus Impactos** (e para quem mais se interessar)<br>
[![DOI](https://zenodo.org/badge/342919527.svg)](https://zenodo.org/badge/latestdoi/342919527)
###### Última atualização: 16 de Junho de 2021.
<br>
Esse notebook foi construido com o objetivo de testar as possibilidades do Geemap para construção de mapas. Não é necessário saber todo o código de cabeça, mas ao menos você pode usar esse notebook para "copiar" algumas soluções quando for necessário.<br>

Use preferencialmente o Jupyter Notebook com o `geemap` e o `ee` instalado.

In [1]:
# importa todas as bibliotecas necessárias
import ee
import geemap
import geemap.colormaps as cm

In [2]:
# A autenticação só é necessária uma vez para usar o ee
#ee.Authenticate()

# iniciar o acesso
ee.Initialize()

In [3]:
# cria um fundo branco com base em uma layer existente (não necessariamente você precisa fazer isso)
blank = ee.Image('NOAA/NGDC/ETOPO1').select('bedrock');
blank = blank.multiply(0) # blank background

In [12]:
# separa um retângulo de interesse. Esse retângulo foi extraido da linha posterior ao mapa com Map.draw_last_feature
AOI = ee.Feature({'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Polygon',
  'coordinates': [[[-84.023438, -36.738884],
    [-84.023438, 13.410994],
    [-33.75, 13.410994],
    [-33.75, -36.738884],
    [-84.023438, -36.738884]]]},
 'properties': {}})

In [38]:
# escolhe o dataset de elevação
elevation = ee.Image('USGS/SRTMGL1_003');

exaggeration = 200; # aqui controla o exagero vertical.
azimuth = 30 # azimute da fonte de luz
zenith = 65 # elevação da fonte de luz é 90 - azimuth
hillshade = ee.Terrain.hillshade(elevation.multiply(exaggeration), azimuth, 90-zenith);

# versão 2 - fazendo um hillshade para o mapa antigo também, com um exagero menor
exaggeration2 = 100; # aqui controla o exagero vertical.
hillshade2 = ee.Terrain.hillshade(elevation.multiply(exaggeration), azimuth, 90-zenith);

fao_gaul = ee.FeatureCollection("FAO/GAUL/2015/level1")
Brasil = fao_gaul.filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))


In [29]:
# cortesia - Um mapa antigo da América do Sul registrado usando georeferencer
# há uma pequena distorção na costa, mas...é o preço da evolução
basemap = ee.Image('users/andrebelem/AmericaDoSul_oldmap')
vizParams = {
  'bands': ['b1', 'b2', 'b3'],
  'min': 0,
  'max': 253,
  'gamma': [0.95, 1.1, 1],
  'opacity': 0.5, # mude a opacidade se desejar. 
};

In [40]:
# Constroi o mapa com base no centro do Brasil. Mude como desejar.

Map = geemap.Map(center=(-12.7261,-54.6657),zoom=4)
Map.add_basemap('Esri Shaded Relief')
Map.addLayer(blank, {'bands': ['bedrock'], 'min': 0, 'max': 0, 'palette': ['white'], 'opacity': 0.7}, 'background')
Map.addLayer(AOI,{'outline_color':'black'},'AOI') # linha ao redor do mapa
Map.addLayer(basemap.clip(AOI),vizParams,'AS')
Map.addLayer(hillshade2.clip(AOI), {'bands': ['hillshade'], 'min': 0, 'max': 255, 'palette': cm.palettes.gray, 'opacity': 0.3}, 'Map Hillshade')

dem_cm = cm.palettes.gist_earth_r

Map.addLayer(Brasil,{'outline_color':'gray','opacity':0.8},'Brasil') # linha ao redor do Brasil

Map.addLayer(elevation.clip(Brasil), {'bands': ['elevation'], 'min': 0, 'max': 1500, 'palette': dem_cm}, 'elevation')
Map.addLayer(hillshade.clip(Brasil), {'bands': ['hillshade'], 'min': 0, 'max': 255, 'palette': cm.palettes.gray, 'opacity': 0.6}, 'ETOPO1 Hillshade')

Map

Map(center=[-12.7261, -54.6657], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [26]:
# para extrair um novo retângulo AOI, desenhe sobre o mapa acima e descomente as linhas abaixo.

#aa = Map.draw_last_feature
#aa.getInfo()